In [ ]:
# IPython magig  tools
%load_ext autoreload
%autoreload 2

from aind_vr_foraging_analysis.utils.parsing import data_access
import aind_vr_foraging_analysis.data_io as data_io
import os
import seaborn as sns

# Plotting libraries
import pandas as pd
import warnings
pd.options.mode.chained_assignment = None  # Ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
import matplotlib.pyplot as plt

foraging_figures = r'C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents'

In [ ]:
date_string = "2024-4-1"
mouse_list = ['754570','754579','754567','754580','754559','754560','754577',
              '754566','754571','754572','754573','754574','754575', 
              '754582','745302','745305','745301',
              
              "715866", "713578", "707349", "716455", 
              "716458","715865","715869","713545","715867",
              "715870","694569", 
              
              '754574', '789914', '789915', '789923', '789917', 
               '789913', '789909', '789910', '789911', '789921', 
               '789918', '789919', '789907', '789903', '789925', 
               '789924', '789926', '789908']

experiment_list = { 1: 'pilot',
                    2 : 'volume_manipulation',
                   3: 'global_reward_rate_patches', 
                   4: 'global_reward_rate_distance_friction',
                   5 : 'learning_reversals'}

In [ ]:
results = []

for mouse in mouse_list:
    print(f"Loading {mouse}...")

    session_paths = data_access.find_sessions_relative_to_date(
        mouse=mouse,
        date_string=date_string,
        when='on_or_after',
    )

    # Determine batch based on mouse number
    mouse_int = int(mouse)
    if mouse == 754574 and date_string >= "2025-4-1":
        batch = 5
    elif mouse_int > 760000:
        batch = 5
    elif mouse_int > 740000:
        batch = 4
    elif mouse_int > 700000:
        batch = 3
    elif mouse_int > 690000:
        batch = 2
    else:
        batch = 1
    session_n = 0
    for session_path in session_paths:
        try:
            all_epochs, stream_data, data = data_access.load_session(session_path)
            reward_sum = all_epochs.loc[all_epochs['is_reward'] == 1, 'reward_amount'].sum()

            results.append({
                'water': reward_sum,
                'length': all_epochs['length'].sum(),
                'mouse': mouse,
                'batch': batch,
                'session_n': session_n
            })
            session_n+=1

        except Exception as e:
            print(f"Error loading {session_path}: {e}")
            continue

# Final DataFrame
cum_df = pd.DataFrame(results)


In [ ]:
cum_df = pd.DataFrame(results)

In [ ]:
test_df = cum_df.groupby(['mouse', 'session_n', 'batch']).length.sum().reset_index()

In [ ]:
sns.lineplot(data=test_df, x='session_n', y='length', hue='batch',  marker='o', lw=2)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', ncol = 3)

In [ ]:
cum_df.to_csv(os.path.join(foraging_figures, f"water_{date_string}.csv"), index=False)

In [ ]:
sns.lineplot(data=cum_df.loc[cum_df.batch!= 2], x='session_n', y='water', hue='batch', palette='Set1')
sns.despine()

In [ ]:
sns.lineplot(data=cum_df.loc[cum_df.batch== 5], x='session_n', y='water', hue='mouse', palette='Set1')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  
sns.despine()